## test excel report export creator

In [1]:
import pandas as pd
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell

In [2]:
# read in test data
# TODO: replace with queryset from django
raw_df = pd.read_csv("test.csv")

raw_df

,id,staff_id,first_name,last_name,branch_name,task_name,punch_time_in,punch_time_out,session_time_hours
0,1112016-01-02 08:00:00,9997,TEST_FIRST_NAME_1,TEST_LAST_NAME_1,TEST_BRANCH_1,TEST_TASK_1,2016-01-02 08:00:00,2016-01-02 11:00:00,3.0
1,1232016-01-03 11:00:00,9997,TEST_FIRST_NAME_1,TEST_LAST_NAME_1,TEST_BRANCH_2,TEST_TASK_3,2016-01-03 11:00:00,2016-01-03 12:00:00,1.0
2,2122016-01-03 10:00:00,9998,TEST_FIRST_NAME_2,TEST_LAST_NAME_2,TEST_BRANCH_1,TEST_TASK_2,2016-01-03 10:00:00,2016-01-03 12:00:00,2.0
3,3112016-01-04 09:00:00,9999,TEST_FIRST_NAME_3,TEST_LAST_NAME_3,TEST_BRANCH_1,TEST_TASK_1,2016-01-04 09:00:00,NaN,NaN


In [3]:
# clean up data types
df = raw_df.copy()
df.punch_time_in = pd.to_datetime(df.punch_time_in)
df.punch_time_out = pd.to_datetime(df.punch_time_out)

print df.dtypes
df

id                            object
staff_id                       int64
first_name                    object
last_name                     object
branch_name                   object
task_name                     object
punch_time_in         datetime64[ns]
punch_time_out        datetime64[ns]
session_time_hours           float64
dtype: object


,id,staff_id,first_name,last_name,branch_name,task_name,punch_time_in,punch_time_out,session_time_hours
0,1112016-01-02 08:00:00,9997,TEST_FIRST_NAME_1,TEST_LAST_NAME_1,TEST_BRANCH_1,TEST_TASK_1,2016-01-02 08:00:00,2016-01-02 11:00:00,3.0
1,1232016-01-03 11:00:00,9997,TEST_FIRST_NAME_1,TEST_LAST_NAME_1,TEST_BRANCH_2,TEST_TASK_3,2016-01-03 11:00:00,2016-01-03 12:00:00,1.0
2,2122016-01-03 10:00:00,9998,TEST_FIRST_NAME_2,TEST_LAST_NAME_2,TEST_BRANCH_1,TEST_TASK_2,2016-01-03 10:00:00,2016-01-03 12:00:00,2.0
3,3112016-01-04 09:00:00,9999,TEST_FIRST_NAME_3,TEST_LAST_NAME_3,TEST_BRANCH_1,TEST_TASK_1,2016-01-04 09:00:00,NaT,NaN


In [4]:
# create new columns
df['volunteer_name'] = df.last_name + ", " + df.first_name
df['month'] = df['punch_time_in'].dt.month
df['year'] = df['punch_time_in'].dt.year

df

,id,staff_id,first_name,last_name,branch_name,task_name,punch_time_in,punch_time_out,session_time_hours,volunteer_name,month,year
0,1112016-01-02 08:00:00,9997,TEST_FIRST_NAME_1,TEST_LAST_NAME_1,TEST_BRANCH_1,TEST_TASK_1,2016-01-02 08:00:00,2016-01-02 11:00:00,3.0,"TEST_LAST_NAME_1, TEST_FIRST_NAME_1",1,2016
1,1232016-01-03 11:00:00,9997,TEST_FIRST_NAME_1,TEST_LAST_NAME_1,TEST_BRANCH_2,TEST_TASK_3,2016-01-03 11:00:00,2016-01-03 12:00:00,1.0,"TEST_LAST_NAME_1, TEST_FIRST_NAME_1",1,2016
2,2122016-01-03 10:00:00,9998,TEST_FIRST_NAME_2,TEST_LAST_NAME_2,TEST_BRANCH_1,TEST_TASK_2,2016-01-03 10:00:00,2016-01-03 12:00:00,2.0,"TEST_LAST_NAME_2, TEST_FIRST_NAME_2",1,2016
3,3112016-01-04 09:00:00,9999,TEST_FIRST_NAME_3,TEST_LAST_NAME_3,TEST_BRANCH_1,TEST_TASK_1,2016-01-04 09:00:00,NaT,NaN,"TEST_LAST_NAME_3, TEST_FIRST_NAME_3",1,2016


In [5]:
grouped = df.groupby(['year','month','volunteer_name'])
volunteer = grouped.agg({'punch_time_in' : 'count', 'session_time_hours' : 'sum'})
volunteer.columns = ['Volunteer Sessions', 'Total Hours']

volunteer

Volunteer Sessions  \
year month volunteer_name                                            
2016 1     TEST_LAST_NAME_1, TEST_FIRST_NAME_1                   2   
           TEST_LAST_NAME_2, TEST_FIRST_NAME_2                   1   
           TEST_LAST_NAME_3, TEST_FIRST_NAME_3                   1   

                                                Total Hours  
year month volunteer_name                                    
2016 1     TEST_LAST_NAME_1, TEST_FIRST_NAME_1          4.0  
           TEST_LAST_NAME_2, TEST_FIRST_NAME_2          2.0  
           TEST_LAST_NAME_3, TEST_FIRST_NAME_3          NaN

In [28]:
volunteer_month = volunteer['Total Hours'].unstack(['year','month'])

volunteer_month

year,2016
month,1
volunteer_name,
"TEST_LAST_NAME_1, TEST_FIRST_NAME_1",4.0
"TEST_LAST_NAME_2, TEST_FIRST_NAME_2",2.0
"TEST_LAST_NAME_3, TEST_FIRST_NAME_3",NaN


In [31]:
grouped = df.groupby(['task_name'])
task = grouped.agg({'punch_time_in' : 'count', 'session_time_hours' : 'sum'})
task.columns = ['Volunteer Sessions', 'Total Hours']

task

,Volunteer Sessions,Total Hours
task_name,,
TEST_TASK_1,2,3.0
TEST_TASK_2,1,2.0
TEST_TASK_3,1,1.0


In [32]:
grouped = df.groupby(['year','month'])
month = grouped.agg({'punch_time_in' : 'count', 'session_time_hours' : 'sum'})
month.columns = ['Volunteer Sessions', 'Total Hours']

month

,,Volunteer Sessions,Total Hours
year,month,,
2016,1,4,6.0


In [35]:
grouped = df.groupby(['year','month','task_name'])
task_month = grouped.agg({'session_time_hours' : 'sum'})
task_month.columns = ['Total Hours']
task_month = task_month.unstack(['year','month'])

task_month

,Total Hours
year,2016
month,1
task_name,
TEST_TASK_1,3.0
TEST_TASK_2,2.0
TEST_TASK_3,1.0


In [7]:
grouped = df.groupby(['year','month','volunteer_name', 'task_name'])
volunteer_task = grouped.agg({'punch_time_in' : 'count', 'session_time_hours' : 'sum'})
volunteer_task.columns = ['Volunteer Sessions', 'Total Hours']
volunteer_task

Volunteer Sessions  \
year month volunteer_name                      task_name                         
2016 1     TEST_LAST_NAME_1, TEST_FIRST_NAME_1 TEST_TASK_1                   1   
                                               TEST_TASK_3                   1   
           TEST_LAST_NAME_2, TEST_FIRST_NAME_2 TEST_TASK_2                   1   
           TEST_LAST_NAME_3, TEST_FIRST_NAME_3 TEST_TASK_1                   1   

                                                            Total Hours  
year month volunteer_name                      task_name                 
2016 1     TEST_LAST_NAME_1, TEST_FIRST_NAME_1 TEST_TASK_1          3.0  
                                               TEST_TASK_3          1.0  
           TEST_LAST_NAME_2, TEST_FIRST_NAME_2 TEST_TASK_2          2.0  
           TEST_LAST_NAME_3, TEST_FIRST_NAME_3 TEST_TASK_1          NaN

### Excel Writer

In [53]:
writer = pd.ExcelWriter("recoverycafe_volunteer_report.xlsx", engine='xlsxwriter')

#####################
#### prints to excel
#####################

# set title format
workbook  = writer.book
title = workbook.add_format()
title.set_font_size(20)
title.set_bold()


#
### summary sheet
#

sheet = 'SUMMARY'

# write tables
month.to_excel(writer, sheet_name=sheet, startrow=3)
task.to_excel(writer, sheet_name=sheet, startrow=3, startcol=5)
task_month.to_excel(writer, sheet_name=sheet, startrow=3, startcol=9)

# set width for columns
worksheet = writer.sheets[sheet]
worksheet.set_column("C:D",18)
worksheet.set_column("F:F",44)
worksheet.set_column("G:H",18)
worksheet.set_column("J:J",44)

# write titles
worksheet.write('A2', 'Totals by Month', title)
worksheet.write('F2', 'Totals by Task', title)
worksheet.write('J2', 'Hours by Task & Month', title)




#
### volunteer sheet
#

sheet = 'Volunteers'

# write tables
volunteer.to_excel(writer, sheet_name=sheet, startrow=3)
volunteer_task.to_excel(writer, sheet_name=sheet, startrow=3, startcol=6)

# set width for columns
worksheet = writer.sheets[sheet]
worksheet.set_column("C:C",44)
worksheet.set_column("D:E",18)
worksheet.set_column("K:L",18)
worksheet.set_column("I:J",44)

# write titles
worksheet.write('A2', 'Volunteers by Month', title)
worksheet.write('G2', 'Volunteers by Task & Month', title)



#
### volunteer by month sheet
#

sheet = 'Volunteers by Month'

# write table
volunteer_month.to_excel(writer, sheet_name=sheet, startrow=3)

# set width for columns
worksheet = writer.sheets[sheet]
worksheet.set_column("A:A",44)

# write titles
worksheet.write('A2', 'Volunteers by Month', title)



#
### raw table sheet
#

sheet = '- Raw Data -'

# write table
raw_df.ix[:,1:].to_excel(writer, sheet_name=sheet)

# set width for columns
worksheet = writer.sheets[sheet]
worksheet.set_column("B:Z",22)



#
### write excel file
#

writer.save()